In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
## Load dataset
data_dir = '/home/hao/Research/apg_data'
data_path = data_dir + "/gmm/gmm_10size"
Data = []
for i in range(10):
    data = torch.from_numpy(np.load(data_path + '/obs_%d.npy' % ((i+1)*10))).float()
    Data.append(data)
K = 3 ## number of clusters
D = 2
NUM_HIDDEN_LOCAL = 32
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

PATH = 'apg-3run-60pts-10samples'
# PATH = 'vae-60pts-10samples'
# PATH = 'apg-sanity2-60pts-10samples'

In [3]:
from model_operations import Init_models
# from model_operations_vae import Init_models

models, optimizer = Init_models(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE, lr=1e-4, RESTORE=True, PATH=PATH)

In [4]:
from testing import Eval
from viz_mc import Viz_MC
from apg import APG_test
from vae import VAE_test

BATCH_SIZE = 5
Viz_INTERVAL = 4
FS = 15
TITLE_FS = 20
OB_MS = 5.0
MU_MARKER = 'X'
MU_MS = 100
OPACITY = 0.3
BOUND = 12
COLORS = ['#AA3377', '#EE7733', '#0077BB', '#009988']
DATA_PTR = 73

In [5]:
visualizer = Viz_MC(K, Viz_INTERVAL, FS, TITLE_FS, OB_MS, OPACITY, BOUND, COLORS)
evaluator = Eval(models, K, D, BATCH_SIZE, CUDA, DEVICE)

# Metrics_sample = evaluator.Test_uniform(APG_test, Data, DATA_PTR, mcmc_steps=12, sample_size=1)
# visualizer.Plot_chains(Metrics_sample['data'], Metrics_sample['samples'], filename='gmm-samples-%d' % DATA_PTR)

In [6]:
# L = 10
# Metrics_converge = evaluator.Test_uniform(APG_test, Data, DATA_PTR, mcmc_steps=99, sample_size=L)
# visualizer.Plot_metrics(Metrics_converge['log_joint'], Metrics_converge['ess'], sample_size=L, filename='gmm-metrics-LL-ESS-%d' % DATA_PTR)

In [7]:
budgets = [[1000, 1], [500, 2], [200, 5], [100, 10], [10, 100], [5, 200]]

LLs= []
ESSs = []
SSs = []
for b in budgets:
    samples = b[0]
    steps = b[1] - 1
    Metrics_converge = evaluator.Test_uniform(APG_test, Data, DATA_PTR, mcmc_steps=steps, sample_size=samples)
    LLs.append(Metrics_converge['log_joint'])
    ESSs.append(Metrics_converge['ess'])
    SSs.append(samples)

In [12]:
LLs[2][0]

tensor([-136.9196, -129.4418,  -87.6638,  -86.2455,  -81.1786])

In [30]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

COLORS = ['#AA3377', '#EE7733', '#0077BB', '#009988', '#CCBB44','#BBBBBB']



def Plot_metrics_v2(fs, LLs, ESSs, SSs, filename):
    num_cols = len(LLs[0])
    gs = gridspec.GridSpec(2, num_cols)
    gs.update(left=0.05 , bottom=0.05, right=0.95, top=0.95, wspace=0.2, hspace=0.05)
    fig = plt.figure(figsize=(fs, fs * 2 / num_cols))
    plt.rc('axes',edgecolor='#eeeeee')
    for col_ind in range(num_cols):
#         ax1 = fig.add_subplot(gs[0, col_ind])
#         ax2 = fig.add_subplot(gs[1, col_ind])
        print('N=%d' % ((col_ind+1)*20))
        for b, LL, in enumerate(LLs):
            ESS = ESSs[b]
            SS = SSs[b]
            print(ESS[col_ind][-1] / SS)
#             ax1.plot(LL[col_ind].data.numpy(), c=COLORS[b], marker='o')
#             ax2.plot(ESS[col_ind].data.numpy() / SS, '--', c=COLORS[b], alpha=0.4)
#     plt.savefig(filename + '.svg', dpi=300)
#     plt.savefig(filename + '.pdf')

In [31]:
fs = 20
Plot_metrics_v2(fs, LLs, ESSs, SSs, filename='test')

N=20
tensor(0.0010)
tensor(0.4938)
tensor(0.7836)
tensor(0.7571)
tensor(0.9915)
tensor(0.9908)
N=40
tensor(0.0030)
tensor(0.6092)
tensor(0.7646)
tensor(0.9494)
tensor(0.9886)
tensor(0.9786)
N=60
tensor(0.0010)
tensor(0.9631)
tensor(0.7952)
tensor(0.9790)
tensor(0.9843)
tensor(0.9965)
N=80
tensor(0.0010)
tensor(0.8888)
tensor(0.9239)
tensor(0.9549)
tensor(0.9841)
tensor(0.9912)
N=100
tensor(0.0010)
tensor(0.9815)
tensor(0.8293)
tensor(0.9715)
tensor(0.9898)
tensor(0.9825)


<Figure size 1440x576 with 0 Axes>